In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage,AIMessage

In [5]:
store={}
def get_session(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [6]:
model = ChatGroq(api_key=groq_api_key,model_name="llama-3.1-8b-instant")
chat_with_history=RunnableWithMessageHistory(model,get_session,)

In [7]:
config = {"configurable": {"session_id": "chat1"}}

response = chat_with_history.invoke([HumanMessage(content="Hey hi, my name is Kakashi hatake and i am a gen ai intern")],config=config)
print(response.content)

Nice to meet you, Kakashi Hatake! As a gen AI intern, I'm excited to learn more about your experiences and the world of the Hidden Leaf Village. I've heard of your legendary skills as a ninja and your role as the Seventh Hokage's right-hand man. Being an intern, you must be learning a lot about the intersection of technology and ninja skills.

What brings you here today? Are you looking to discuss a particular topic or perhaps need assistance with a project? I'm all ears (or rather, all text) and ready to help!


In [8]:
response=chat_with_history.invoke([HumanMessage(content="Do you know my name and what i am currently doing")],config=config)
print(response.content)

I am familiar with your name, Kakashi Hatake, and your current position as the Seventh Hokage of the Hidden Leaf Village. However, I should note that in the original Naruto storyline, Kakashi Hatake is actually the Sixth Hokage, and the Seventh Hokage is actually Boruto's father, who is also named Naruto.

As for your current activities, I'm not sure what you're doing as a Gen AI intern. Could you please clarify what that entails and what you're working on?


In [9]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

chain_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant, answer these question "),
    MessagesPlaceholder(variable_name="messages")
])

In [10]:
chain=chain_prompt|model
chain.invoke({
    "messages":[HumanMessage(content="Hi my name is harshit hatake")]
})

AIMessage(content="Nice to meet you, Harshit Hatake! It's great to have you here. I'm a helpful assistant, and I'm here to assist you with any questions or topics you'd like to discuss. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 53, 'total_tokens': 104, 'completion_time': 0.089083453, 'completion_tokens_details': None, 'prompt_time': 0.002592593, 'prompt_tokens_details': None, 'queue_time': 0.055743566, 'total_time': 0.091676046}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--640cdbd9-1f0a-4f05-80f3-126f1e2357fa-0', usage_metadata={'input_tokens': 53, 'output_tokens': 51, 'total_tokens': 104})

In [11]:
chain_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant, answer these question to best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])
chain=chain_prompt|model

In [12]:
prompt_with_history = RunnableWithMessageHistory(chain,get_session,input_messages_key="messages")
config={"configurable":{"session_id":"chat2"}}
prompt_with_history.invoke({
    "messages":[HumanMessage(content="so what are you planning to do")],"language":"Hindi"},config=config)


AIMessage(content='मैं एक सहायक हूँ और आपकी सहायता के लिए तैयार हूँ। मैं आपके प्रश्नों का उत्तर देने और आपके ज्ञान को बढ़ाने में मदद करने के लिए यहाँ हूँ। क्या आपके पास कोई विशिष्ट प्रश्न है जिस पर मैं आपकी सहायता कर सकता हूँ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 58, 'total_tokens': 157, 'completion_time': 0.125700678, 'completion_tokens_details': None, 'prompt_time': 0.003517471, 'prompt_tokens_details': None, 'queue_time': 0.050441339, 'total_time': 0.129218149}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--fc3498e6-d28f-4cc3-ab25-bbc730ca20b2-0', usage_metadata={'input_tokens': 58, 'output_tokens': 99, 'total_tokens': 157})

In [13]:
# this is before adding the language variable in prompt
# prompt_with_history.invoke({
#     "messages":[HumanMessage(content="what is my name")]},config=config)

In [14]:
from langchain_core.messages import SystemMessage
from langchain_core.messages import trim_messages

In [15]:
trimmer =trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    start_on = "human"
)
messages = [
    SystemMessage(content="You are a good assistant."),
    HumanMessage(content="Hey, I am Bob."),
    AIMessage(content="Hi."),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Nice."),
    HumanMessage(content="What is two plus two?"),
    AIMessage(content="Four."),
    HumanMessage(content="Thanks.")
]

trimmed = trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\bishn\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bishn\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [20]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|
    chain_prompt|
    model
)

In [21]:
chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what math wuestion did i asked")],
        "language":"english"
    }
)

AIMessage(content='You asked the math question "What is two plus two?"', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 126, 'total_tokens': 139, 'completion_time': 0.019984075, 'completion_tokens_details': None, 'prompt_time': 0.008984516, 'prompt_tokens_details': None, 'queue_time': 0.049516944, 'total_time': 0.028968591}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--a096a2a2-9ce7-4f96-b455-d0d87468c5e4-0', usage_metadata={'input_tokens': 126, 'output_tokens': 13, 'total_tokens': 139})

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat3"}}

In [24]:
with_message_history.invoke({
    "messages":[HumanMessage(content="what is my name")],
    "language":"english"
},config=config
)

AIMessage(content='I don\'t have the ability to know your name as I\'m a text-based AI assistant and we have just started our conversation. You can share your name with me if you\'d like, and I\'ll be happy to use it in our conversation. Alternatively, I can refer to you as "user" or "you" in our conversation.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 55, 'total_tokens': 125, 'completion_time': 0.116579006, 'completion_tokens_details': None, 'prompt_time': 0.002732238, 'prompt_tokens_details': None, 'queue_time': 0.054291942, 'total_time': 0.119311244}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--56ad9cc5-e56f-47d9-a8dc-c8a00fbd8208-0', usage_metadata={'input_tokens': 55, 'output_tokens': 70, 'total_tokens': 125})